# ROC曲線の実装

このノートブックでは、ROC曲線の理論から実装、可視化まで詳しく学習します。

## 学習目標
- ROC曲線の理論的理解
- 手動実装とscikit-learnの比較
- 実データでのROC曲線描画
- ROC曲線の解釈と活用


In [ ]:
# 必要なライブラリのインポート
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, roc_auc_score
import warnings
warnings.filterwarnings('ignore')

# 日本語フォントの設定
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_style('whitegrid')


## 1. ROC曲線の手動実装


In [ ]:
def calculate_roc_manual(y_true, y_pred_proba):
    """
    ROC曲線の手動実装
    
    Parameters:
    y_true: 実際のラベル
    y_pred_proba: 予測確率
    
    Returns:
    fpr, tpr, thresholds: ROC曲線の座標
    """
    # 閾値の取得（降順）
    thresholds = np.sort(np.unique(y_pred_proba))[::-1]
    tpr_list, fpr_list = [], []
    
    for threshold in thresholds:
        y_pred = (y_pred_proba >= threshold).astype(int)
        
        # TP, FP, FN, TNの計算
        tp = np.sum((y_true == 1) & (y_pred == 1))
        fp = np.sum((y_true == 0) & (y_pred == 1))
        fn = np.sum((y_true == 1) & (y_pred == 0))
        tn = np.sum((y_true == 0) & (y_pred == 0))
        
        # TPRとFPRの計算
        tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
        fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
        
        tpr_list.append(tpr)
        fpr_list.append(fpr)
    
    return np.array(fpr_list), np.array(tpr_list), thresholds

# 簡単な例でROC曲線を理解
np.random.seed(42)
y_true_example = np.random.binomial(1, 0.5, 100)
y_pred_proba_example = np.random.beta(2, 2, 100)

# 手動実装とscikit-learnの比較
fpr_manual, tpr_manual, _ = calculate_roc_manual(y_true_example, y_pred_proba_example)
fpr_sklearn, tpr_sklearn, _ = roc_curve(y_true_example, y_pred_proba_example)

# 可視化
plt.figure(figsize=(15, 5))

# 手動実装
plt.subplot(1, 3, 1)
plt.plot(fpr_manual, tpr_manual, 'b-', linewidth=2, label='Manual Implementation')
plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve (Manual)')
plt.legend()
plt.grid(True, alpha=0.3)

# scikit-learn
plt.subplot(1, 3, 2)
plt.plot(fpr_sklearn, tpr_sklearn, 'r-', linewidth=2, label='scikit-learn')
plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve (scikit-learn)')
plt.legend()
plt.grid(True, alpha=0.3)

# 比較
plt.subplot(1, 3, 3)
plt.plot(fpr_manual, tpr_manual, 'b-', linewidth=2, label='Manual')
plt.plot(fpr_sklearn, tpr_sklearn, 'r--', linewidth=2, label='scikit-learn')
plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Comparison')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("=== ROC曲線の基本概念 ===")
print("TPR (True Positive Rate) = TP / (TP + FN) = Recall")
print("FPR (False Positive Rate) = FP / (FP + TN) = 1 - Specificity")
print(f"手動実装とscikit-learnの一致: {np.allclose(fpr_manual, fpr_sklearn, atol=1e-10)}")


## 2. 実データでのROC曲線


In [ ]:
# 乳がんデータセットの読み込み
cancer = load_breast_cancer()
X, y = cancer.data, cancer.target

# データの分割と標準化
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# モデルの訓練
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_train_scaled, y_train)
y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]

# ROC曲線の計算
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

# 可視化
plt.figure(figsize=(10, 8))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Breast Cancer Dataset')
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)
plt.show()

print(f"=== 乳がんデータセットでのROC曲線 ===")
print(f"AUC: {roc_auc:.4f}")
print(f"データセットの情報:")
print(f"  サンプル数: {len(y_test)}")
print(f"  クラス分布: {np.bincount(y_test)}")
print(f"  正例の割合: {np.mean(y_test):.2%}")

# 異なる閾値での性能
def evaluate_thresholds(y_true, y_pred_proba, thresholds):
    results = []
    for threshold in thresholds:
        y_pred = (y_pred_proba >= threshold).astype(int)
        tp = np.sum((y_true == 1) & (y_pred == 1))
        fp = np.sum((y_true == 0) & (y_pred == 1))
        fn = np.sum((y_true == 1) & (y_pred == 0))
        tn = np.sum((y_true == 0) & (y_pred == 0))
        
        tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
        fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
        
        results.append({
            'threshold': threshold,
            'tpr': tpr,
            'fpr': fpr
        })
    
    return results

# 閾値の評価
thresholds_eval = np.arange(0.1, 1.0, 0.1)
results_eval = evaluate_thresholds(y_test, y_pred_proba, thresholds_eval)

print(f"\n=== 異なる閾値での性能 ===")
for result in results_eval:
    print(f"閾値: {result['threshold']:.1f}, TPR: {result['tpr']:.3f}, FPR: {result['fpr']:.3f}")


## まとめ

**学習した内容**：
- ROC曲線の理論と実装
- 手動実装とscikit-learnの比較
- 実データでのROC曲線描画

**重要なポイント**：
- ROC曲線は閾値に依存しない評価
- AUCは0から1の範囲
- 左上に近いほど良いモデル
